In [3]:
import argparse
import io
import os

import h5py
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf
from keras import backend as K
from keras.layers import Input, Lambda
from keras.models import Model

from mobiledet.models.keras_yolo import (preprocess_true_boxes,
                                     yolo_eval, yolo_head, yolo_loss)
from mobiledet.utils.draw_boxes import draw_boxes

In [7]:
from mobiledet.models.keras_darknet19 import darknet19

image_input = Input(shape=(416, 416, 3))
base_model = darknet19(image_input)
base_model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 416, 416, 3)       0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 416, 416, 32)      864       
_________________________________________________________________
batch_normalization_19 (Batc (None, 416, 416, 32)      128       
_________________________________________________________________
leaky_re_lu_19 (LeakyReLU)   (None, 416, 416, 32)      0         
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 208, 208, 32)      0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 208, 208, 64)      18432     
_________________________________________________________________
batch_normalization_20 (Batc (None, 208, 208, 64)      256       
__________